# Predict regulatory regions from the DNA sequence using keras

This notebook is similar to the keras tutorial, but instead of using the Janggu Dataset objects directly, we shall use the built-in Sequence class. This approach is also possible with the new keras and tensorflow (v2) interface.

In [1]:
import os

import numpy as np
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import Maximum

from pkg_resources import resource_filename

from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import Reverse, Complement
from janggu.data import JangguSequence

from sklearn.metrics import roc_auc_score

Using TensorFlow backend.
/home/wkopp/anaconda3/envs/jdev1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wkopp/anaconda3/envs/jdev1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wkopp/anaconda3/envs/jdev1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wkopp/anaconda3/envs

In [2]:
from IPython.display import Image

np.random.seed(1234)

In [3]:
import keras, tensorflow

tensorflow.__version__, keras.__version__

('1.14.0', '2.2.5')

First, we need to specify the output directory in which the results are stored.

In [4]:
os.environ['JANGGU_OUTPUT'] = '/home/wkopp/janggu_examples'

Specify the DNA sequence feature order. Order 1, 2 and 3 correspond to mono-, di- and tri-nucleotide based features (see Tutorial).

In [5]:
order = 3

In [6]:
# load the dataset
# The pseudo genome represents just a concatenation of all sequences
# in sample.fa and sample2.fa. Therefore, the results should be almost
# identically to the models obtained from classify_fasta.py.
REFGENOME = resource_filename('janggu', 'resources/pseudo_genome.fa')
# ROI contains regions spanning positive and negative examples
ROI_TRAIN_FILE = resource_filename('janggu', 'resources/roi_train.bed')
ROI_TEST_FILE = resource_filename('janggu', 'resources/roi_test.bed')
# PEAK_FILE only contains positive examples
PEAK_FILE = resource_filename('janggu', 'resources/scores.bed')

Load the datasets for training and testing

In [7]:
# Training input and labels are purely defined genomic coordinates
DNA = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                   roi=ROI_TRAIN_FILE,
                                   binsize=200,
                                   order=order,
                                   cache=True)

# The ReduceDim wrapper transforms the dataset from a 4D object to a 2D table-like representation
LABELS = ReduceDim(Cover.create_from_bed('peaks', roi=ROI_TRAIN_FILE,
                               bedfiles=PEAK_FILE,
                               binsize=200,
                               resolution=200,
                               cache=True,
                               storage='sparse'))


DNA_TEST = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                        roi=ROI_TEST_FILE,
                                        binsize=200,
                                        order=order)

LABELS_TEST = ReduceDim(Cover.create_from_bed('peaks',
                                    bedfiles=PEAK_FILE,
                                    roi=ROI_TEST_FILE,
                                    binsize=200,
                                    resolution=200,
                                    storage='sparse'))


In [8]:
print('training set:', DNA.shape, LABELS.shape)
print('test set:', DNA_TEST.shape, LABELS_TEST.shape)

training set: (7797, 198, 1, 64) (7797, 1)
test set: (200, 198, 1, 64) (200, 1)


Specify a keras model with compatible dimesions input and output dimensions for the example.

In [9]:
xin = Input((200 - order + 1, 1, pow(4, order)))

# use layer to scan both strands of the DNA
convlayer = Conv2D(30, (21, 1), activation='relu')

# first the forward strand
forward = convlayer(xin)
# then the reverse complemented sequence
reverse = convlayer(Complement()(Reverse()(xin)))

#merge the results by using the maximum activation wrt the strands
layer = Maximum()([forward, reverse])

layer = GlobalAveragePooling2D()(layer)
output = Dense(1, activation='sigmoid')(layer)

model = Model(xin, output)

model.compile(optimizer='adadelta', loss='binary_crossentropy',
              metrics=['acc'])
model.summary()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 198, 1, 64)   0                                            
__________________________________________________________________________________________________
reverse_1 (Reverse)             (None, 198, 1, 64)   0           input_1[0][0]                    
__________________________________________________________________________________________________
complement_1 (Complement)       (None, 198, 1, 64)   0           reverse_1[0][0]                  
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 178, 1, 30)   40350       input_1[0][0]              

In [10]:
trainseq = JangguSequence(DNA, LABELS, as_dict=False)

In [11]:
hist = model.fit(trainseq, epochs=100)

print('#' * 40)
print('loss: {}, acc: {}'.format(hist.history['loss'][-1],
                                 hist.history['acc'][-1]))
print('#' * 40)


Epoch 1/100
244/244 [==============================] - 10s 40ms/step - loss: 0.6216 - acc: 0.6472
Epoch 2/100
244/244 [==============================] - 9s 35ms/step - loss: 0.5521 - acc: 0.7284
Epoch 3/100
244/244 [==============================] - 9s 35ms/step - loss: 0.5313 - acc: 0.7427
Epoch 4/100
244/244 [==============================] - 12s 49ms/step - loss: 0.4979 - acc: 0.7717
Epoch 5/100
244/244 [==============================] - 15s 61ms/step - loss: 0.4840 - acc: 0.7726
Epoch 6/100
244/244 [==============================] - 15s 61ms/step - loss: 0.4699 - acc: 0.7812
Epoch 7/100
244/244 [==============================] - 15s 63ms/step - loss: 0.4639 - acc: 0.7855
Epoch 8/100
244/244 [==============================] - 15s 61ms/step - loss: 0.4352 - acc: 0.8015
Epoch 9/100
244/244 [==============================] - 15s 60ms/step - loss: 0.4258 - acc: 0.8076
Epoch 10/100
244/244 [==============================] - 16s 64ms/step - loss: 0.4179 - acc: 0.8139
Epoch 11/100
244/244

244/244 [==============================] - 9s 36ms/step - loss: 0.0430 - acc: 0.9898
Epoch 83/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0410 - acc: 0.9899
Epoch 84/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0414 - acc: 0.9899
Epoch 85/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0396 - acc: 0.9908
Epoch 86/100
244/244 [==============================] - 9s 35ms/step - loss: 0.0390 - acc: 0.9913
Epoch 87/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0386 - acc: 0.9900
Epoch 88/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0383 - acc: 0.9907
Epoch 89/100
244/244 [==============================] - 9s 37ms/step - loss: 0.0368 - acc: 0.9912
Epoch 90/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0368 - acc: 0.9912
Epoch 91/100
244/244 [==============================] - 9s 36ms/step - loss: 0.0355 - acc: 0.9918
Epoch 92/100
244/244 [===========

The predictions may be converted back to Cover object and subsequently exported as bigwig in order to inspect the plausibility of the results in the genome browser.

In [12]:
testseq = JangguSequence(DNA_TEST, as_dict=False)

# convert the prediction to a cover object
pred = model.predict(testseq)
cov_pred = Cover.create_from_array('BindingProba', pred, LABELS_TEST.gindexer)

# predictions (or feature activities) can finally be exported to bigwig
cov_pred.export_to_bigwig(output_dir=os.environ['JANGGU_OUTPUT'])


In [13]:
pred

array([[9.99151051e-01],
       [9.75318134e-01],
       [9.96574104e-01],
       [9.95460033e-01],
       [9.96464014e-01],
       [8.64547014e-01],
       [9.96426702e-01],
       [9.99921978e-01],
       [9.89611328e-01],
       [9.98907030e-01],
       [9.35466111e-01],
       [9.86055791e-01],
       [9.26077247e-01],
       [9.66469288e-01],
       [9.70292747e-01],
       [9.98694539e-01],
       [9.99305248e-01],
       [9.97891068e-01],
       [9.81406808e-01],
       [9.99887824e-01],
       [9.92626667e-01],
       [9.98514771e-01],
       [9.99931455e-01],
       [9.99961138e-01],
       [9.94316578e-01],
       [9.97490048e-01],
       [8.59833419e-01],
       [9.17471051e-01],
       [9.73466277e-01],
       [9.76478577e-01],
       [9.99038696e-01],
       [9.97492373e-01],
       [8.55449438e-01],
       [9.97820377e-01],
       [8.67090762e-01],
       [7.52588272e-01],
       [9.99957919e-01],
       [9.38101947e-01],
       [9.83132601e-01],
       [9.99170959e-01],


In [14]:
print("AUC:", roc_auc_score(LABELS_TEST[:], pred))

AUC: 0.9947
